# **<font size=7>Lab 3 Report (Group 29)</font>**

<u>รายชื่อสมาชิกในกลุ่ม:</u>
- นายธนัช ธรนิศตระกูล 63340500025
- นายสิรวิชญ์ ลั้วสมบูรณ์ 63340500066

<hr>

## **<font size=6>ขั้นตอนการลงมือทำ</font>**

<u>**<font size=4>การแสดงผลแบบจำลองของหุ่นยนต์ใน "rviz2"**</font></u>

**ภายใน package "sandevistan_description"**

**NOTE**: Folder "meshes" จะมีไฟล์ mesh (.stl) สำหรับการสร้างแบบจำลองของหุ่นยนต์

1. เขียนไฟล์ Xacro สำหรับสร้าง Macro ต่าง ๆ เพื่อใช้สร้างหุ่นยนต์ (create_link.xacro)
    - Macro สำหรับใส่สีให้หุ่นยนต์
    
    ![Color Macro](pic/colormacro.png)

    - Macro สำหรับสร้าง Mesh ของหุ่นยนต์

    ![Color Macro](pic/meshmacro.png)

    - Macro สำหรับสร้าง Visual ของหุ่นยนต์

    ![Color Macro](pic/vismacro.png)

    - Macro สำหรับเพิ่ม Collision ของหุ่นยนต์

    ![Color Macro](pic/colmacro.png)

    - Macro สำหรับเพิ่ม Inertia ของหุ่นยนต์

    ![Color Macro](pic/inertiamacro.png)

    - Macro สำหรับสร้าง Link ของหุ่นยนต์

    ![Color Macro](pic/linkmacro.png)

2. เขียนไฟล์ Xacro ชื่อ "sandevistan_robot.urdf.xacro" และนำไปไว้ที่ Folder "sandevistan_description/xacro_description" โดยด้านในไฟล์จะมีการสร้าง Link ทุกลิงค์ที่มี (รวมถึง World และ End Effector)

    - Link 0: เป็นลิงค์ของฐาน (Base Link) จะเชื่อมกับ World และ Link 1
    
    ![XACRO Base Link](pic/xacrobase.png)

    - Link 1: เป็นลิงค์แรกจากฐาน มีการขยับเป็น Revolute จะเชื่อมกับ Link 0 และ Link 2

    ![XACRO Link 1](pic/xacrolink1.png)

    - Link 2: เป็นลิงค์ที่ 2 จากฐาน มีการขยับเป็น Revolute จะเชื่อมกับ Link 1 และ Link 3

    ![XACRO Link 2](pic/xacrolink2.png)

    - Link 3: เป็นลิงค์ที่ 3 จากฐาน มีการขยับเป็น Revolute จะเชื่อมกับ Link 2 และ End Effector

    ![XACRO Link 3](pic/xacrolink3.png)

3. gazebo.xacro

4. include.xacro

5. เขียนไฟล์ Launch ไว้สำหรับ Launch Package (description launch)

**ภายใน Package "sandevistan_kinematics"**

**Note:** ก่อนเริ่มคำนวณจะต้องรอ 10 วินาที เพื่อให้ Gazebo เปิดก่อนที่จะเริ่มการคำนวณ

ใน Package นี้จะมีการคำนวณหาตำแหน่งของ Joint ผ่าน Inverse Kinematics โดยจะมีการรับค่าจาก IMU เข้ามาเพื่อนำมาบังคับให้หุ่นยนต์ขยับตามทิศทางที่ต้องการ
- เมื่อ X เป็นบวก หุ่นยนต์จะยืดไปด้านหน้า
- เมื่อ X เป็นลบ หุ่นยนต์จะหดไปด้านหลัง
- เมื่อ Y เป็นบวก หุ่นยนต์จะหมุนทวนเข็มนาฬิกา
- เมื่อ Y เป็นลบ หุ่นยนต์จะหมุนตามเข็มนาฬิกา
- เมื่อ Z เป็นบวก หุ่นยนต์จะยืดไปด้านบน
- เมื่อ Z เป็นลบ หุ่นยนต์จะหดไปด้านล่าง

<hr>

## **<font size=6>การเชื่อมต่อ IMU เข้ากับ Xicro</font>**

Arduino Uno Wifi Rev2 จะรับค่า Linear Acceleration และ Angular Velocity ในแกน X, Y, Z จาก IMU และส่งค่าไปยัง Xicro Node ผ่าน UART Protocol จากนั้น Xicro Node จะทำการส่งค่าไปยัง kinematic_server Node (Algorithm สำหรับควบคุมหุ่นยนต์) และ calibrate Node (Node สำหรับทำ IMU Calibration) ผ่าน ROS Topic ที่ชื่อว่า /sandevistan_IMU ด้วยความถี่ 100 Hz

การเขียนไฟล์ Arduino ชื่อ "" สำหรับส่งค่าไปยัง Xicro Node

![Arduino Code](pic/arduinocode.png)

 

<hr>

## **<font size=6>การทำ Calibration ของ IMU</font>**

ในการทำ Calibration จะเริ่มจากการสร้าง Node ชื่อ "calibrate" โดยใน Node นี้จะมีการรับค่าจาก IMU ผ่าน Topic ชื่อ "/sandevistan_IMU" จากนั้นจะนำค่าที่ได้มาเก็บไว้ใน List จากนั้นหลังจากผ่านไป 10 วินาที จะนำค่าที่เก็บไว้มาหาค่า Mean และเก็บเข้าสู่ไฟล์ YAML เพื่อนำไปใช้ใน Kinematic Server ต่อไป

- สร้าง Node ชื่อ "calibrate"

![Calibrate Node](pic/calibrateNode.png)

- รับค่าจาก IMU ผ่าน Topic ชื่อ "/sandevistan_IMU"

![Get IMU](pic/getIMU.png)

- นำค่าที่ได้มาเก็บไว้ใน List

![Store IMU](pic/storeIMU.png)

- หลังจากผ่านไป 10 วินาที จะนำค่าที่เก็บไว้มาหาค่า Mean และเก็บเข้าสู่ไฟล์ YAML เพื่อนำไปใช้ใน Kinematic Server ต่อไป

![Mean Function](pic/meanFunction.png)

![Spin Node](pic/spinNode.png)

![Write YAML](pic/writeYAML.png)

<hr>

## **<font size=6>Algorithm X สำหรับสั่งงานหุ่นยนต์</font>**

1. kinematic_server Node จะรับค่า IMU ผ่าน ROS Topic ที่ชื่อว่า /sandevistan_IMU

2. 

<hr>

## **<font size=6>การนำไฟล์ URDF/XACRO ไปยังโปรแกรม Gazebo</font>**